In [1]:
import numpy as np
from qiskit_experiments.library import StandardRB, InterleavedRB
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
import qiskit.circuit.library as circuits

In [2]:
from qiskit import assemble
from qiskit_ibm_provider import IBMProvider
#from qiskit import IBMQ
from qiskit import execute
from qiskit.tools import job_monitor

In [3]:
IBMProvider.save_account('', overwrite=True)
IBMProvider()

<IBMProvider>

In [4]:
provider = IBMProvider(instance="ibm-q-ncsu/nc-state/vlsi-cad-tools")
backend = provider.get_backend('ibm_hanoi')

In [5]:
lengths_x_qubit = np.arange(1, 10, 3)
lengths_1_qubit = np.arange(1, 800, 200)
num_samples = 10
seed = 1010
qubits = (0, 1,2,3,4,5)

# Run a 1-qubit RB experiment on qubits 1 to determine the error-per-gate of 1-qubit gates
single_exps = BatchExperiment(
    [
        StandardRB([qubit], lengths_1_qubit, num_samples=num_samples, seed=seed)
        for qubit in qubits
    ],
    flatten_results=True,
)
expdata_1q = single_exps.run(backend).block_for_results()

In [ ]:
exp_xq.circuits()[29].decompose().depth()

In [ ]:
exp_xq.circuits()[0].draw('mpl')

In [ ]:
print(c.decompose().depth())
c.decompose().draw("mpl")